<a href="https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_stage_2_21x21x10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""ali_build_model_stage_2_MobileNetV2_aug_1000_dense.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wSVEmcoWzRm-7B6UGkHB8ysfMDRT_xK5
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls 'drive/My Drive/geo/'

geo_Unet_colab.ipynb		Untitled1.ipynb
geo_Unet.ipynb			Untitled2.ipynb
test2.npy			upload_sample.csv
test_submit_0831_2.csv		water_first_try_2_2.h5
test_submit_0831_3.csv		water_first_try_2.h5
test_submit_0831_ch32.csv	water_first_try_2_prepro_ch16.h5
test_submit_0831_ch32_gram.csv	water_first_try_2_prepro_ch32_gram.h5
test_submit_0831_ch64_gram.csv	water_first_try_2_prepro_ch32.h5
test_submit_0831.csv		water_first_try_2_prepro_ch64_gram.h5
test_submit.csv			water_first_try_2_prepro.h5
train_data.npy			water_first_try_2_prepro_twoloss.h5
train_labels.npy		water_first_try_3_prepro.h5
Untitled0.ipynb			water_first_try.h5


In [0]:
src=  'drive/My Drive/geo/'

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, \
Conv2DTranspose,BatchNormalization
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers

In [9]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)

(4200, 21, 21, 20)
(4200, 20, 20, 1)


# Reduce channel from 21 to 11

In [13]:
index_list = list(range(20))
select_list = [idx for idx in index_list if idx % 2  == 0 ]
select_list



[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [15]:
data = data[:,:,:,select_list]
data.shape

(4200, 21, 21, 10)

In [0]:
data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )

In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=8):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((21,21,10))

    conv0 = Conv2D(32,(2,2),padding='valid')(inputs)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    model.summary()
    
    return model
    
    

In [55]:
model = build_model_1()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 8)
poo12 (?, 5, 5, 16)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 32)
poo14 (?, 1, 1, 64)
conv7 (?, 4, 4, 32)
conv7 (?, ?, ?, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 21, 21, 10)   0                                            
__________________________________________________________________________________________________
conv2d_87 (Conv2D)              (None, 20, 20, 32)   1312        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_88 (Conv2D)              (None, 20, 20, 8)    2312        conv2d_87[0][0]                  
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 20

In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [57]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
    

    if os.path.isfile(src+'water_first_try.h5'):

      model = load_model(src+'water_first_try.h5')
      print ('model laoded')
    else:
      print ('model not exist')
else:
  print ('not to use model')


not to use model


In [58]:





checkpoint = ModelCheckpoint(filepath=src+'geo_stage2_first_try.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=30)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=15, min_lr=0.00001,verbose=1)
callback_list = [checkpoint, earlystop,reduce_lr]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=1000, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)

Train on 2688 samples, validate on 672 samples
Epoch 1/1000
2688/2688 [==============================] - 4s 2ms/step - loss: 0.9681 - mean_absolute_error: 0.7785 - val_loss: 0.7037 - val_mean_absolute_error: 0.6665
Epoch 2/1000
2688/2688 [==============================] - 3s 940us/step - loss: 0.6196 - mean_absolute_error: 0.6245 - val_loss: 0.5419 - val_mean_absolute_error: 0.5809
Epoch 3/1000
2688/2688 [==============================] - 3s 983us/step - loss: 0.4952 - mean_absolute_error: 0.5547 - val_loss: 0.4689 - val_mean_absolute_error: 0.5376
Epoch 4/1000
2688/2688 [==============================] - 3s 980us/step - loss: 0.4154 - mean_absolute_error: 0.5056 - val_loss: 0.3707 - val_mean_absolute_error: 0.4769
Epoch 5/1000
2688/2688 [==============================] - 3s 975us/step - loss: 0.3540 - mean_absolute_error: 0.4649 - val_loss: 0.3331 - val_mean_absolute_error: 0.4510
Epoch 6/1000
2688/2688 [==============================] - 3s 1ms/step - loss: 0.3153 - mean_absolute_erro

In [0]:
model.evaluate(val_x,val_y)

with open('train_data.pickle', 'wb') as f:
    pickle.dump(train_data, f)

with open('train_data.pickle', 'rb') as f:
    train_data = pickle.load(f)

print (train_data.shape)    


In [0]:
def shuffle_list(s,l):
    final_list = []
    n_list = np.arange(l)
    np.random.shuffle(n_list)
    b_s = int(np.floor(l/s))
    #print (b_s)
    
    for i in range (b_s):
        print (i)
        final_list.append(n_list[i*s:(i+1)*s])
    return final_list

In [0]:
l = len(train_x)
s= 32
batch_list = shuffle_list(s,l)
for batch in bacth_list:
    X = train_x[batch]
    Y = train_y[batch]
    model.train_on_batch(X,Y)
    print (np.argmax(model.predict(X),axis=-1))


NameError: name 'train_x' is not defined